<a href="https://colab.research.google.com/github/LoiueFragosoUwUr/Sentiment-analysis-for-tourist-reviews-AI/blob/main/MeIA_Analisis_de_Sentimientos_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Configuración**

Si está ejecutando este notebook en Google Colab, ejecute la siguiente celda para instalar las bibliotecas que necesitamos:

In [ ]:
#!pip install transformers datasets
!pip install transformers==4.28.0 datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00


**Cargar dataset**

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA/MeIA_Rest_Mex_Sentiment_Analysis_2023_Train.xlsx', index_col=0)
#data.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Análizis de datos del conjunto de entrenamiento original.

In [ ]:
print("Número de instacias: ",len(data))
print("Distribución de instancias por clase:")
print(data["Class"].value_counts())

Número de instacias:  17500
Distribución de instancias por clase:
5    5250
4    4375
3    3500
2    2625
1    1750
Name: Class, dtype: int64


In [ ]:
data.head()

,Class
Review,
Dar mantenimiento adecuado y aclarar jurisdicción de quién es. De uno de los puentes que lleva a las cascadas cayó una turista 100 m hacia abajo en abril 2021 (obviamente falleció al caer sobre las rocas) y nunca se aclaró si la responsabilidad fue de CFE o del Municipio.,1
"yo estaba tan emocionada de este viaje, pero tras el 2-1 2 horas de autobús /viaje y 1 parada de un parque ecológico con un Cenote - me di cuenta este viaje era más de una trampa para turistas que un sitio histórico. Las ruinas en sí tienen más vendedores ladrando y haciendo ruido con sus mercancías que puede debilitar un palo. que siga usted y su guía y tratar de vender a usted mientras usted está tratando de escuchar al guía sobre lo que estás viendo. Hacía calor, y el poquito de sombra disponible fue tomado por los vendedores. La única cosa que he oído en ese viaje fue ""oferta! oferta! más barato que Wal-Mart! Casi gratis!"" que necesitan para mantener a los vendedores en una zona y para permitir a los visitantes que se acerquen a ellos, no sentimos acosados y seguido por ellos. Y por amor de Dios, acondicionar algunos lugares sombreados para sentarse allí!!! Había basura en las ruinas. . Nunca recomendaría este viaje a cualquiera. Fue un día entero y $ nunca volver. Una de las ""maravillas"" del mundo? Realmente? sólo me"" maravilla"" ¿por qué me dejaron lo que podría ser un sitio guay conseguir así crapped!.",1
"Este hotel es un engaño. De cuatro estrellas nada. Una estrella siendo generoso. Carece de higiene básica, el servicio pésimo, las habitaciones viejas, cama rota, polvo en todos lados, el desayuno impresentable, manteles sucios, sin servilletas, un desastre. Nunca más.",1
"La ducha se inundo completamente a los 5 min de usarla por una coladera sucia y el agua se transfirio al resto del baño. No habia toallas tampoco. Lo que habla de la poca higiene y poco cuidado a los detalles. El personal ni siquiera se disculpo por el incidente. Los ruidos en el pasillo se oyen muchísimo a la habitación. Mala experiencia y una noche por hab. Estandar costo casi 2000 pesos mexicanos, La decoracion se ve vieja y anticuada.",1
"Decidimos alquilar un taxi por un día para ir a Habana. El taxista recomendó y nos llevó a este restaurante para el almuerzo. Creo que él hizo esto como un montón de grupos de turistas ir allí. Para dos adultos y un niño de 7...y mi hijo de 9 años pagamos casi $150.00 canadiense para el almuerzo! No nos dimos cuenta los ""lados"" eran extra. La camarera Camino venta sugestiva ... sabiendo que los chicos es probable que se llena en el plato principal. Además, el 10% de propina era incluidos. La carne rallado que los niños lo estaba bien y abundante, pero no vale la pena los 18 dólares un plato. No podía comer la porción de langosta de mi comida como la langosta probado ... skunky ... creo que era un tipo diferente de langosta más barato que era asqueroso. Este fue un servicio de almuerzo, así que no había música en vivo ... quizás leer otras críticas, es donde este lugar es bueno para. La única cosa era la sabrosa bebida ... caña de azúcar y zumo de piña ... lo que estoy seguro de que podríamos conseguir por la mitad del precio en otro sitio. Fuera de todo nuestro viaje a Cuba, este restaurante fue la única decepción y empeoramiento del viaje! Casi pagado tanto para el almuerzo como nosotros pagamos el taxista que nos llevó desde Valedaro para todo el día!Más",1


**División estratificada**

Se divide estratificadamente el dataset original para formar un conjunto de entrenamiento y pruebas.

In [ ]:
from sklearn.model_selection import train_test_split
a,b = train_test_split(data, test_size=0.30, stratify = data['Class'])
print("Longitud del conjunto de entrenamiento (70%): ",len(a))
print("Longitud del conjunto de pruebas (30%): ",len(b))
print(len(a)+len(b))

Longitud del conjunto de entrenamiento (70%):  12250
Longitud del conjunto de pruebas (30%):  5250
17500


In [ ]:
print("Distribución de instancias del conjunto de entrenamiento por clase:")
print(a["Class"].value_counts())

Distribución de instancias del conjunto de entrenamiento por clase:
5    3675
4    3063
3    2450
2    1837
1    1225
Name: Class, dtype: int64


In [ ]:
from datasets import Dataset, concatenate_datasets, DatasetDict

ds_train = Dataset.from_pandas(a)
ds_test = Dataset.from_pandas(b)
ds_dict = {'train' : ds_train, 'test':ds_test}
dataset = DatasetDict(ds_dict)
dataset

DatasetDict({
    train: Dataset({
        features: ['Class', 'Review'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['Class', 'Review'],
        num_rows: 5250
    })
})

**Pre-procesamiento del dataset**

Cambiar el nombre de la columana que contiene las clases a predecir por "labels"

In [ ]:
dataset = dataset.rename_column("Class", "labels")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'Review'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['labels', 'Review'],
        num_rows: 5250
    })
})

Las etiquetas (labels) en HF se inicializan a partir del numeró 0, por lo cuál, las etiquetas de polaridad se deben modifican de la siguiente manera: label 1 = 0, label 2 = 1, label 3 = 2, label 4 = 3 y label 5 = 4.

In [ ]:
def polarity_init(example):
  example["labels"] = example["labels"] - 1
  return example

dataset = dataset.map(polarity_init)

dataset.set_format("pandas")
df = dataset["train"][:]
print(df["labels"].value_counts())
dataset.reset_format()

Map:   0%|          | 0/12250 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

4    3675
3    3063
2    2450
1    1837
0    1225
Name: labels, dtype: int64


**Login en Hugging Face para compartir el modelo**

In [ ]:
# Esto sólo funciona en Google Colab! Para los notebooks normales, es necesario ejecutar esto en el terminal
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

Si no tienes instalado Git LFS, puedes hacerlo descomentando y ejecutando la celda de abajo:

In [ ]:
!apt install git-lfs
!git config --global user.email "vg055@hotmail.com"
!git config --global user.name "Victor055"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


**Tokenizar las reseñas**

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.vocab_size

119547

In [ ]:
def tokenize_reviews(examples):
    return tokenizer(examples["Review"], truncation=True)

columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

Map:   0%|          | 0/12250 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5250
    })
})

In [ ]:
encoded_dataset["train"][0]

{'labels': 1,
 'input_ids': [101,
  86405,
  19086,
  12477,
  10110,
  10109,
  15641,
  10104,
  10811,
  12368,
  10104,
  10182,
  99702,
  193,
  60404,
  27830,
  10570,
  28644,
  117,
  11266,
  10183,
  10139,
  25635,
  11462,
  10121,
  10549,
  10192,
  10196,
  10125,
  13101,
  119,
  27512,
  10119,
  41148,
  11482,
  10104,
  78781,
  10121,
  64325,
  13471,
  10115,
  10285,
  67444,
  10104,
  16186,
  14147,
  26088,
  10229,
  193,
  71280,
  193,
  10125,
  16921,
  18121,
  41516,
  119,
  119,
  119,
  16507,
  117,
  11266,
  60917,
  10104,
  36584,
  29291,
  10192,
  10196,
  24256,
  18345,
  10121,
  10119,
  15189,
  24693,
  51127,
  11359,
  10220,
  32730,
  21898,
  119,
  10224,
  24693,
  10411,
  73974,
  12368,
  10192,
  12758,
  10543,
  10104,
  10186,
  10110,
  10125,
  18121,
  193,
  11161,
  45094,
  10310,
  56588,
  193,
  10347,
  84146,
  119,
  10150,
  22866,
  10543,
  14002,
  117,
  10119,
  12282,
  10104,
  41574,
  10104,
  15

**Cargar el modelo pre-entrenado**

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'pre_class

**Definir metríca de evaluación**

In [ ]:
from datasets import load_metric

metric = load_metric("f1")
metric

<ipython-input-21-ee0dc6f5e028>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


Metric(name: "f1", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    labels (`list` of `int`): The set of labels to include when `average` is not set to `'binary'`, and the order of the labels if `average` is `None`. Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority negative class. Labels not present in the data will result in 0 components in a macro average. For multilabel targets, labels are column indices. By default, all labels in `predictions` and `references` are used in sorted order. Defaults to None.
    pos_label (`int`): The class to be considered the positive class, in the case where `average` is set to `binary`. Defaults to 1.
    average (`string`): This parameter is required for multiclass/multilabel targets. If set to `None`, the sco

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

**Afinar el modelo pre-entrenado (Fine-Tuning)**

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=2
train_dataset = encoded_dataset["train"]
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-MeIA-AnalisisDeSentimientos"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1469: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case vg055/distilbert-base-multilingual-cased-finetuned-MeIA-AnalisisDeSentimientos).
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

Cloning https://huggingface.co/vg055/distilbert-base-multilingual-cased-finetuned-MeIA-AnalisisDeSentimientos into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/516M [00:00<?, ?B/s]

Download file runs/Jun16_23-13-19_cfce56d978d8/events.out.tfevents.1686957215.cfce56d978d8.1137.0: 100%|######…

Download file runs/Jun16_23-13-19_cfce56d978d8/1686957215.3004065/events.out.tfevents.1686957215.cfce56d978d8.…

Download file runs/Jun28_17-07-10_bf6dd9edfaf5/events.out.tfevents.1687972168.bf6dd9edfaf5.380.0: 100%|#######…

...

Clean file runs/Jun29_01-28-49_d48e365ce337/events.out.tfevents.1688003201.d48e365ce337.200.2: 100%|##########…

Clean file pytorch_model.bin:   0%|          | 1.00k/516M [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-8c1e32eec3ac>", line 3, in <cell line: 3>
    trainer = Trainer(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 550, in __init__
    self.init_git_repo(at_init=True)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3497, in init_git_repo
    self.repo = Repository(self.args.output_dir, clone_from=repo_name, token=self.args.hub_token)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 118, in _inner_fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/repository.py", line 516, in __init__
    self.clone_from(repo_url=clone_from)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 118, 

TypeError: ignored

In [ ]:
trainer.train()

**Evaluación del modelo**

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.0305339097976685, 'eval_f1': 0.5444967854287687, 'eval_runtime': 49.5333, 'eval_samples_per_second': 105.989, 'eval_steps_per_second': 6.642, 'epoch': 2.0}


**Subir el modelo a Hugging Face**

In [ ]:
trainer.push_to_hub()

Upload file runs/Jun16_23-13-19_cfce56d978d8/events.out.tfevents.1686957215.cfce56d978d8.1137.0: 100%|########…

Upload file runs/Jun16_23-13-19_cfce56d978d8/events.out.tfevents.1686958139.cfce56d978d8.1137.2: 100%|########…

To https://huggingface.co/vg055/distilbert-base-multilingual-cased-finetuned-MeIA-AnalisisDeSentimientos
   5e131dc..ef3afdb  main -> main

   5e131dc..ef3afdb  main -> main

To https://huggingface.co/vg055/distilbert-base-multilingual-cased-finetuned-MeIA-AnalisisDeSentimientos
   ef3afdb..74094e1  main -> main

   ef3afdb..74094e1  main -> main



'https://huggingface.co/vg055/distilbert-base-multilingual-cased-finetuned-MeIA-AnalisisDeSentimientos/commit/ef3afdb209025b778e44058cda3dd009a13caed4'

**Guarda modelo y utilizarlo desde disco local**

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA")

In [ ]:
from transformers import AutoModelForSequenceClassification
model2 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA/")

In [ ]:
print(model2)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import pipeline
tx = pipeline("text-classification", model=model2, tokenizer=tokenizer)
texto = "Bueno es un. Hotel que ya es necesario un mantenimiento y una modernización urgente porque con el nivel que esta presentando a hora no le alcanza ni par un 3 estrella sus ascensores son deplorable  y tiene un mal servicio de su personal"
tx(texto)

**Pruebas**

Cargar conjunto de datos de pruebas

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA/MeIA_Rest_Mex_Sentiment_Analysis_2023_Test.xlsx', index_col=0)

Cargar pipeline del modelo entrenado

In [ ]:
from transformers import pipeline
pi = pipeline("text-classification", model="vg055/distilbert-base-multilingual-cased-finetuned-MeIA-AnalisisDeSentimientos", max_length=512, truncation=True)

**Ejemplo de prueba**

In [ ]:
text = "Bueno es un. Hotel que ya es necesario un mantenimiento y una modernización urgente porque con el nivel que esta presentando a hora no le alcanza ni par un 3 estrella sus ascensores son deplorable  y tiene un mal servicio de su personal"
res = pi(text)
print(res)

**Conjunto de pruebas y generción de archivo de texto para reto MeIA**

In [ ]:
#Abrir archivo de texto en modo escritura para almacenar los resultados para el reto
with open('Resultados_MeIATeam_ModeloBase.txt', 'w') as f:
  for i in range(len(data)):
      print(i," - ",data.iloc[i]['Review'])
      res = pi(data.iloc[i]['Review'])
      label = res[0]['label']
      #Establecer la etiqueta correspondiente
      if label == 'LABEL_0':
        label = "1"
      elif label == 'LABEL_1':
        label = "2"
      elif label == 'LABEL_2':
        label = "3"
      elif label == 'LABEL_3':
        label = "4"
      elif label == 'LABEL_4':
        label = "5"
      #Escribir predicción del sentimiento en el archivo de texto con el formaro correspondiente
      f.write(str(i)+"\t"+label+"\n")
      #Imprimir en pantalla lo que se escribio en el archivo txt
      print(str(i)+"\t"+label+"\n")
      #Descomente si solo quiere probar este bloque de codigo para las primeras tres instancias de pruebas
      #if i ==2:
        #break

In [ ]:
import numpy as np
import matplotlib as plt
#initialize
# a list
List1=[]

num=1
numcube= num**3

while numcube <100:
    #We can add elements to a list by using append
    List1.append(numcube)
    num += 1
    numcube = num**3

print (List1)

print(List1[0],List1[-1])

List2=[]

for num in List1:
    num-=50
    List2.append(num)
print(List2)
#we can create an array from an already existing list
array1 = np.array(List1)
#their visualization is slightly different

print("array representation",array1)
print("list representation",List1)
#numpy functions can take lists or array as inputs      list as input and array as outputs
# but the will return an array as output

res1= np.sin(List1)
res2= np.sin(array1)

print(res1)
print(res2)
#create a matrix with a numpy array and lists of lists
list3=[[1,0],[0,-1]]
mat1= np.array(list3)
print(mat1)
print(list3)
#arrays behave like vectors
#np.dot allows us to do matrix multiplication
vector1= np.array([.6,-.3])
vector2= np.dot(mat1,vector1)

print(vector2)
print(np.dot(vector2,vector1))

xOR= 10
xOB=-10
xO =[xOR,xOB]

vOR=0
vOB=1.8
#we will pack them with a numpy array
#this will help us with the matrix vector multiplication
vO= np.array([vOR,vOB])
#mR = redmarblemass
#mB = bluemarblemass
#extra variables
mR = 3
mB = 3
msum=mR+mB
mdiff= mR-mB
Mass_matrix= np.array([[mdiff/msum,2*mB/msum],[2*mR/msum,-mdiff/msum]])
vT= np.dot(Mass_matrix,vO)


print("velocity of the red marble after the collision",vT[0])
print("velocity of the blue marble after the collision",vT[1])

coll_time =(xO[0]-xO[1])/(vO[1]-vO[0])
print("Collision at time =", coll_time," sec")
#use the function to calculate the position as a function of time

def position(xOO,vOO,t):
    #determine the position t seconds after being at xOO,
    #while traveling with velocity vOO
    return xOO+vOO*t
coll_pos = position(xO,vO,coll_time)
coll_pos
Tsteps=100
time = np.linspace(0,2*coll_time,Tsteps)
red_pos=[]
blue_pos=[]
# loop iterating time value
for t in time:
    #before collision we use initial values of position and velocity
    if t< coll_time:
        red_pos=position(xOR,vOR,t)
        blue_pos=position(xOR,vOB,t)
#after collision we use the calculated values of position and velocity
#time is now the difference between total and the collision time
    else:
        red_pos_t=position(coll_pos[0],vT[0],t-coll_time)
        blue_pos_t= position(coll_pos[1],vT[1],t-coll_time)
    red_pos.append(red_pos_t)
    blue_pos.append(blue_pos_t)
plt.plot(time,red_pos,'r')
plt.plot(time,blue_pos,'r')
plt.title('One dimensional collision')
plt.xlabel('time(seconds)')
plt.ylabel('position(cm)')
plt.legend(['Red marble','Blue marble'])
plt.tight_layout(pad=0.4,v_pad=6.5,h_pad=1.0)

[1, 8, 27, 64]
1 64
[-49, -42, -23, 14]
array representation [ 1  8 27 64]
list representation [1, 8, 27, 64]
[0.84147098 0.98935825 0.95637593 0.92002604]
[0.84147098 0.98935825 0.95637593 0.92002604]
[[ 1  0]
 [ 0 -1]]
[[1, 0], [0, -1]]
[0.6 0.3]
0.27
velocity of the red marble after the collision 1.8
velocity of the blue marble after the collision 0.0
Collision at time = 11.11111111111111  sec


AttributeError: ignored